In [11]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld

In [12]:
sys.path.append('..')

In [13]:
from modules.transforms import *

In [14]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [15]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [16]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [17]:
pop=pd.read_sql_table('producto1_Covid_19', schema='producto1',con=cnx) 
pop=pop.groupby('Region')['Poblacion'].sum()
popTOT=pop.sum()

In [18]:
popTOT

19458310.0

In [19]:
#FALLECIDOS DEIS
s="DEIS"
n="DEIS_2020"
DEIS=pd.read_sql_table(n, con=cnx,schema=s)
DEIS=DEIS[DEIS['COVID'].notnull()]
master=DEIS.groupby('FECHA_DEF')['index'].count().reset_index()
master.columns=['Fecha','Fallecidos DEIS Diarios']
master=master.merge(DEIS[DEIS['COVID']=='Covid PCR+'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (PCR+)'}),on='Fecha')
master=master.merge(DEIS[DEIS['COVID']=='Covid Sospechoso'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (Covid Sospechoso)'}),on='Fecha')
master['Fallecidos DEIS (PCR+) Acumulado']=master['Fallecidos DEIS Diarios (PCR+)'].cumsum()
master['Fallecidos DEIS (Covid Sospechoso) Acumulado']=master['Fallecidos DEIS Diarios (Covid Sospechoso)'].cumsum()
master['Fallecidos DEIS Acumulado']=master['Fallecidos DEIS Diarios'].cumsum()
master['Fallecidos DEIS (PCR+) x Millón/Hab.']=master['Fallecidos DEIS (PCR+) Acumulado']/(popTOT/1000000)
master['Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.']=master['Fallecidos DEIS (Covid Sospechoso) Acumulado']/(popTOT/1000000)
master['Fallecidos DEIS x Millón/Hab.']=master['Fallecidos DEIS Acumulado']/(popTOT/1000000)

In [20]:
master

,Fecha,Fallecidos DEIS Diarios,Fallecidos DEIS Diarios (PCR+),Fallecidos DEIS Diarios (Covid Sospechoso),Fallecidos DEIS (PCR+) Acumulado,Fallecidos DEIS (Covid Sospechoso) Acumulado,Fallecidos DEIS Acumulado,Fallecidos DEIS (PCR+) x Millón/Hab.,Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.,Fallecidos DEIS x Millón/Hab.
0,2020-03-20,2,1,1,1,1,2,0.051392,0.051392,0.102784
1,2020-03-21,2,1,1,2,2,4,0.102784,0.102784,0.205568
2,2020-03-25,2,1,1,3,3,6,0.154176,0.154176,0.308352
3,2020-03-26,2,1,1,4,4,8,0.205568,0.205568,0.411135
4,2020-03-27,4,1,3,5,7,12,0.256960,0.359743,0.616703
...,...,...,...,...,...,...,...,...,...,...
145,2020-08-15,90,48,42,10645,4015,14660,547.067037,206.338577,753.405614
146,2020-08-16,83,45,38,10690,4053,14743,549.379674,208.291470,757.671144
147,2020-08-17,96,52,44,10742,4097,14839,552.052054,210.552715,762.604769
148,2020-08-18,74,39,35,10781,4132,14913,554.056339,212.351432,766.407771


In [24]:
#casos activos según fecha de inicio de síntomas
s="producto53"
n="producto53_confirmados_"
nuevos=pd.read_sql_table(n, con=cnx,schema=s)

In [27]:
pd.DataFrame(nuevos.groupby('fecha')['Promedio'].sum())

,Promedio
fecha,
2020-03-01,20.470
2020-03-02,20.835
2020-03-03,22.315
2020-03-04,24.075
2020-03-05,27.295
...,...
2020-09-01,2158.485
2020-09-02,2267.100
2020-09-03,2243.680


In [26]:
nuevos

,level_0,confirmados.1,confirmados.2,confirmados.3,confirmados.4,confirmados.5,confirmados.6,confirmados.7,confirmados.8,confirmados.9,...,confirmados.200,Promedio,Min,Max,0.5,0.95,servicio.salud,fecha,index,region
0,0,0,0,0,1,1,0,1,0,0,...,0,0.340,0.0,3.0,0.00,2.0,151011,2020-03-01,0,15
1,1,0,1,0,0,1,1,0,1,1,...,0,0.255,0.0,4.0,0.00,1.0,151011,2020-03-02,1,15
2,2,2,0,1,2,1,1,0,0,0,...,1,0.285,0.0,3.0,0.00,1.0,151011,2020-03-03,2,15
3,3,0,0,0,0,1,0,0,0,1,...,0,0.345,0.0,3.0,0.00,2.0,151011,2020-03-04,3,15
4,4,0,0,0,0,3,0,0,0,0,...,0,0.335,0.0,3.0,0.00,2.0,151011,2020-03-05,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5098,5098,61,80,58,77,70,63,72,79,83,...,100,73.810,38.0,120.0,73.00,102.7,81012,2020-09-01,5098,9
5099,5099,44,54,66,46,52,47,58,60,69,...,99,65.970,31.0,111.0,65.97,93.9,81012,2020-09-02,5099,9
5100,5100,39,49,41,65,68,41,41,76,71,...,76,60.890,23.0,118.0,59.00,88.9,81012,2020-09-03,5100,9
5101,5101,25,32,62,77,63,62,52,59,72,...,85,58.720,25.0,115.0,58.00,85.0,81012,2020-09-04,5101,9
